In [ ]:
# !pip install pytorch_transformers
# !pip install pymystem3

In [ ]:
import re
import numpy as np
import pandas as pd

from sklearn import decomposition
import matplotlib.pyplot as plt

import torch
from pytorch_transformers import BertTokenizer, BertConfig
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import random
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

In [ ]:
for index in range(1,2):
    dataset = []

    with open(f'vox{index}.txt', newline='\n', mode='r+', encoding='utf-8') as f:
        for line in f:
            dataset.append(line.replace('\r\n', ''))

    names = ['МЭТТ', 'ТРЭВИС', 'ТАЛЕСИН', 'МАРИША', 'ОРИОН', 'ЛОРА', 'ЛИАМ', 'СЭМ', 'ЗАК']

    dataset
    text = ' '.join(dataset)
    for i in names:
        text = text.replace(i, '\n'+i)

    file_vectors = open(f'vox{index}_t.txt', "w+", encoding='utf-8')
    for i in text.split('\n'):
        file_vectors.write(i)
        file_vectors.write('\n')

    text_by_sentence = []
    for i in text.split('\n'):
        if i == '':
            continue
        t = i.replace('...', '#')
        t = t.replace('--', '$')
    #     t = re.split('; |\. |! |\? |# |$ ', t)
        r = re.findall('.*?[.!\?#$]"? ?', t)
        if r == []:
            r = [t]
        name = r[0].split(':')[0]
        r[0] = r[0].replace('#', ' ... ')
        r[0] = r[0].replace('$', ' -- ')
        r[0] = name + ': ' + (':'.join(r[0].split(':')[1:])).strip()
        r[0] = r[0].replace('  ', ' ')
        if len(r) > 1:
            for j, v in enumerate(r[1:]):
                r[j+1] = v.replace('#', ' ... ')
                r[j+1] = r[j+1].replace('$', ' -- ')
                r[j+1] = r[j+1].strip()
                r[j+1] = name + ': ' + r[j+1]
                r[j+1] = r[j+1].replace('  ', ' ')
        text_by_sentence += r

#     for i,v in enumerate(text_by_sentence):
#         text_by_sentence[i] = text_by_sentence[i].replace('#', ' ... ')
#         text_by_sentence[i] = text_by_sentence[i].replace('$', ' -- ')
#         text_by_sentence[i] = text_by_sentence[i].strip()

    file_vectors = open(f'vox{index}_sentenced.txt', "w+", encoding='utf-8')
    for i in text_by_sentence:
        file_vectors.write(i.strip())
        file_vectors.write('\n')
    file_vectors.close()
        
    file_vectors2 = open(f'vox{index}_sentenced_extra.txt', "w+", encoding='utf-8')
    print(file_vectors)
    for i in text_by_sentence[4400:]:
        print(i.strip())
        file_vectors2.write(i.strip())
        file_vectors2.write('\n')
    file_vectors2.close()


In [ ]:
df = []
with open(f'vox1_sentenced_weight.txt', newline='\n', mode='r+', encoding='utf-8') as f:
    for line in f:
        df.append(line.replace('\r\n', ''))
df = [[i.split(': ')[0], ': '.join(i.split(': ')[1:])[:-1], i[-1:]] for i in df]
df = pd.DataFrame(df, columns=['Actor', 'Text', 'Class'])
df

In [ ]:
len(df[df.Class == '0']), len(df[df.Class == '1']), len(df[df.Class == '2'])

In [ ]:
name_dict = {'МЭТТ':0, 'ТРЭВИС':1, 'ТАЛЕСИН':2, 'МАРИША':3, 'ОРИОН':4, 'ЛОРА':5, 'ЛИАМ':6, 'СЭМ':7, 'ЗАК':8}
name_dict_column = ['Matt', 'Trevis', 'Talesin', 'Marisha', 'Orion', 'Lora', 'Liam', 'Sam', 'Zack']
ls = df['Actor'].tolist()
ls = [name_dict[i] for i in ls]
ls_c = [[0]*9 for i in ls]
for i,v in enumerate(ls):
    ls_c[i][v] = 1
for i,v in enumerate(name_dict_column):
    df[name_dict_column[i]] = [ls_c[j][i] for j,k in enumerate(ls_c)]
    
ls = df['Class'].tolist()
ls_c = [[0]*3 for i in ls]
for i,v in enumerate(ls):
    ls_c[i][int(v)] = 1
for i,v in enumerate(['Nongaming', 'Metagaming', 'Gaming']):
    df[v] = [ls_c[j][i] for j,k in enumerate(ls_c)]

df.to_csv('vox1_stock_data.csv', encoding = 'cp1251',sep=',')

df = pd.read_csv('vox1_stock_data.csv', encoding = 'cp1251',sep=',')
df = df.drop(["Unnamed: 0"], axis=1)
df

In [ ]:
texts_length = [len(i.split()) for i in df.Text.tolist()]
min_value, max_value = min(texts_length), max(texts_length)

X = [i for i in range(min_value, max_value + 1)]
Y = [(texts_length.count(value) / len(texts_length)) for value in range(min_value, max_value + 1)]

plt.bar(X, Y)
plt.show()

In [ ]:
max_text_length = 20
k = sum([1 for i in texts_length if i <= max_text_length]) / len(texts_length)
print(f'max_text_length <= {max_text_length} = {k}')

In [ ]:
df_base = df.copy()
for i in [',', ':', ';', ' – ']:
    df_base.Text = df_base.Text.str.replace(i, ' ')
df_base.Text = df_base.Text.str.replace('...', '%')
df_base.Text = df_base.Text.str.replace('--', '#')
for i in ['!', '?', '"', '.', '(', ')']:
    df_base.Text = df_base.Text.str.replace(i, ' '+i+' ')

df_base.Text = df_base.Text.str.replace('ё', 'е')

df_base.Text = df_base.Text.str.strip()
df_base.Text = df_base.Text.str.lower()
df_base.Text = df_base.Text.str.replace('   ', ' ')
df_base.Text = df_base.Text.str.replace('  ', ' ')

a = re.sub('  ', ' ', re.sub('  ', ' ', re.sub(r'[^а-яa-z ]', '', ' '.join(df_base.Text)))).strip()
m = pymystem3.Mystem()
res = [m.lemmatize(word)[0] for word in word_tokenize(a)]

v_tf = torch.zeros((len(df_base),len(unique)))
for i,v in enumerate(df_base.Text):
    for j in v.split():
        v_tf[i][unique.index(j)] = v.count(j)/len(v.split())

v_idf = torch.zeros((1,len(unique)))
for i,v in enumerate(unique):
    n = 0
    for j,k in enumerate(df_base.Text):
        if v in k.split():
            n += 1
    v_idf[0][i] = math.log10(len(unique) / n)
v_idf

v_tfidf = torch.mul(v_tf, v_idf)

df_tfidf.to_csv('vox1_tf_idf_unlemm.csv', encoding = 'cp1251',sep=',')

In [ ]:
pca = decomposition.PCA(n_components=20)
pca.fit(df_tfidf[unique])
pcaed = pca.transform(df_tfidf[unique])

df_tfidf_pca = pd.read_csv('vox1_tf_idf_unlemm_pca.csv', encoding = 'cp1251',sep=',')
df_tfidf_pca = df_tfidf_pca.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
df_tfidf_pca

In [ ]:
df_bert = df.copy()
df_bert.Text = '[CLS] '+df_bert.Text+' [SEP]'
df_bert

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
X = df_tfidf_pca[name_dict_column+[str(i) for i in range(20)]]
y = df_tfidf_pca[['Nongaming', 'Metagaming', 'Gaming']]
X = torch.tensor(X.values)
y = torch.tensor(y.values).reshape(4385,3)

In [ ]:
X_test = torch.tensor(X[2177:2867,:].tolist() + X[3921:4365,:].tolist()).float()
y_test = torch.tensor(y[2177:2867,:].tolist() + y[3921:4365,:].tolist()).float()
X_train = torch.tensor(X[:2177].tolist() + X[2867:3921].tolist() + X[4365:].tolist()).float()
y_train = torch.tensor(y[:2177].tolist() + y[2867:3921].tolist() + y[4365:].tolist()).float()
X_test.shape, X_train.shape, y_test.shape, y_train.shape

In [ ]:
X_train = X_train.reshape((-1, X_train.shape[1], 1))
X_test = X_test.reshape((-1, X_test.shape[1], 1))
X_test.shape, X_train.shape, y_test.shape, y_train.shape

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self, hidden_dim, batch_size, inputs_count, label_size, num_stacked_layers):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_stacked_layers = num_stacked_layers

        self.lstm = nn.LSTM(inputs_count, hidden_dim, num_stacked_layers, batch_first=True)
#         self.seq = nn.Sequential(nn.Sigmoid(),
        self.linear = nn.Linear(hidden_dim, label_size)
                                

    def forward(self, x):
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_dim)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_dim)
        x, _ = self.lstm(x, (h0, c0))
        x = self.linear(x[:, -1, :])
#        x = self.seq(x[:, -1, :])
        return x

    def forward_test(self, x):
        self.lstm(x)
        x = self.linear(x[:, -1, :])
        return x
    

In [ ]:
label_size = 3
inputs_count = 1 #X_train.shape[1]
hidden_dim = 64
batch_size = 50
epochs = 10000
num_stacked_layers = 1


'''use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    model.cuda()'''

data_train = TensorDataset(X_train, y_train)
data_test = TensorDataset(X_test, y_test)
dataloader_train = DataLoader(data_train, batch_size=batch_size, shuffle=False, drop_last=True)
dataloader_test = DataLoader(data_test, batch_size=batch_size, shuffle=False, drop_last=True)

model = LSTMClassifier(hidden_dim, batch_size, inputs_count, label_size, num_stacked_layers)    
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)

In [ ]:
for epoch in range(epochs):
    for X_batch, y_batch in dataloader_train:
        pred = model(X_batch)
        loss = loss_function(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ls.append(loss.item())
        
    model.train(False)
    running_loss = 0.0
    
    for X_batch, y_batch in dataloader_test:
        with torch.no_grad():
            pred = model.forward(X_batch)
            loss = loss_function(pred, y_batch)
            running_loss += loss
            
    avg_loss_across_batches = running_loss / len(dataloader_test)
    print(avg_loss_across_batches)
#     test_pred = model.forward_test(X_test)
#     accuracy = (test_pred.argmax(dim=1) == y_test).float().mean()
#     print(accuracy)

In [ ]:
tokenizer = BertTokenizer,from_pretrained('bert-base-uncase', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in df_bert.Text]

In [ ]:
for epoch in range(epochs):
    for X_batch, y_batch in dataloader_train:
        pred = model(X_batch)
        loss = loss_function(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ls.append(loss.item())
        
    model.train(False)
    running_loss = 0.0
    
    for X_batch, y_batch in dataloader_test:
        with torch.no_grad():
            pred = model.forward(X_batch)
            loss = loss_function(pred, y_batch)
            running_loss += loss
            
    avg_loss_across_batches = running_loss / len(dataloader_test)
    print(avg_loss_across_batches)
#     test_pred = model.forward_test(X_test)
#     accuracy = (test_pred.argmax(dim=1) == y_test).float().mean()
#     print(accuracy)